In [20]:
import numpy as np
from Bio.Align import substitution_matrices as sm


def calc_dp(a, b, S, d=-5):
    n, m = len(a)+1, len(b)+1
    f = np.zeros(shape=(n, m))
    
    for i in range(1, n):
        for j in range(1, m):
            f[i, j] = max(f[i-1, j] + d, f[i, j-1] + d, f[i-1, j-1] + S[a[i-1], b[j-1]], 0)
    return f


def calc_path(f, a, b, d=-5):
    n, m = len(a), len(b)
    
    def rec(i, j, state_a, state_b):
        if f[i, j] == 0:
            yield state_a[::-1], state_b[::-1]
            return
        if i > 0 and j > 0 and f[i, j] == f[i-1, j-1] + S[a[i-1], b[j-1]]:
            yield from rec(i-1, j-1, state_a + a[i-1], state_b + b[j-1])
        if i > 0 and f[i, j] == f[i-1, j] + d:
            yield from rec(i-1, j, state_a + a[i-1], state_b + '-')
        if j > 0 and f[i, j] == f[i, j-1] + d:
            yield from rec(i, j-1, state_a + '-', state_b + b[j-1])
    
    idx = np.argmax(f)
    x = idx // len(f[0])
    y = idx % len(f[0])
    return rec(x, y, '', '')

In [21]:
for name in ['BLOSUM45', 'PAM30']:
    print('Matrix %s:' % name)
    S = sm.load(name)
    for a in ['ARNDCQ', 'RNDCQE', 'NDC']:
        for b in ['ARNDCQ', 'RNDCQE']:
            f = calc_dp(a, b, S)
            print(a, b, ': score =', np.max(f))
            for p in calc_path(f, a, b):
                print(p)
            print()
    print()

Matrix BLOSUM45:
ARNDCQ ARNDCQ : score = 43.0
('ARNDCQ', 'ARNDCQ')

ARNDCQ RNDCQE : score = 38.0
('RNDCQ', 'RNDCQ')

RNDCQE ARNDCQ : score = 38.0
('RNDCQ', 'RNDCQ')

RNDCQE RNDCQE : score = 44.0
('RNDCQE', 'RNDCQE')

NDC ARNDCQ : score = 25.0
('NDC', 'NDC')

NDC RNDCQE : score = 25.0
('NDC', 'NDC')


Matrix PAM30:
ARNDCQ ARNDCQ : score = 48.0
('ARNDCQ', 'ARNDCQ')

ARNDCQ RNDCQE : score = 42.0
('RNDCQ', 'RNDCQ')

RNDCQE ARNDCQ : score = 42.0
('RNDCQ', 'RNDCQ')

RNDCQE RNDCQE : score = 50.0
('RNDCQE', 'RNDCQE')

NDC ARNDCQ : score = 26.0
('NDC', 'NDC')

NDC RNDCQE : score = 26.0
('NDC', 'NDC')


